In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore

c:\Projects\medical-chatbot-llama2\medical-chatbot-using-llama2\env\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV')

In [3]:
index_name = "medical-chatbot"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [4]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
query = "Tell me about fever?"
vectorstore.similarity_search(query)

[Document(page_content='tion, an abscess which drains foul-smelling pus, and\npossibly fever . Symptoms for specific infections are\nas follows:\n• Tooth and gum infections. Swollen, tender bleeding\ngums, bad breath , and pain . Severe infections may\nproduce oozing sores.\n• Throat infection. An extremely sore throat , bad breath,\na bad taste in the mouth, fever, and a sense of choking .\n• Lung infection. Chest pain, coughing, difficulty breath-\ning, fever, foul-smelling sputum, and weight loss.\n• Intraabdominal infection. Pain, fever, and possibly, if fol-\nlowing surgery, foul- smelling drainage from the wound.\n• Pelvic infection. Foul-smelling pus or blood draining\nfrom the uterus, general or localized pelvic pain, fever,and chills.\n• Skin and soft tissue infection. Infected wounds are red,\npainful, swollen, and may drain a foul-smelling pus.Skin infection causes localized swelling, pain, redness,and possibly a painful, open sore (ulcer) which drainsfoul-smelling pus. Seve

In [5]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [6]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [7]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q8_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':384,
                          'temperature':0.8})

In [8]:
# # Load model directly
# from transformers import AutoModel
# model = AutoModel.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML")

In [7]:
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever() 
)  
qa.invoke(query) 

{'query': 'Tell me about fever?',
 'result': " The symptoms (sFAQ:\nI don's(sAnaer\nI don's  Helpful Hint infection.docaracteriaer\nI don's 1: ana\nI don's 31 person, what are available at the following organisms that can you know about Anaer'sThe helpful answersHelpful answersAnswered Step 12 years ago. which can i loved On Jun 777723 months ago\nAnnaer's\nI don’s5 weeks .\nI don's(s0 Helpful AnsweredIn some questions.\nI don's \nI don's\nYou have a specific anaer 1AnswerExplanarians, but it's0Helpful An anaer:Thank you are available in 9/feedback to Questions 2 months ago\nI don's\nYou have a possible causes of Anaer. Able organisms which type infection and Expertous organisms that will befor the question at the following symptoms, what kind of anaerWhat are just a specific types of this question about Anaer:\nDon's (10 Helpful Answered • Follow \nI don's(sThe patient’sAnaer ."}

In [9]:
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs 
)  
qa.invoke(query) 

{'query': 'Tell me about fever?',
 'result': "Thank you can tell us understand that drauglytically produced by bacteria bacteria bacter, ana) ___________________________________________________________________________________________________________________________________________________________________________________________________________________________An anaerobic microorganisms present infeo you'I cannot be able bacteria bacteria bacteria bacter, I apologize the patient’microbacteria bacteria bacteria bacteria bacter(3.\nAnaerobes.\nI can help@user'microorganisms, I cannot\nANAnaerobic organism.\nAn ana) Yes,\nThe symptoms,\nYes,\nAn anaerobes or more detail .\nAn anaerobic bacteria bacteria bacteria bacteria bacteria bacterobes and soft microorganisms present bacter that will dependsonal microorganisms,\nI apologize.",
 'source_documents': [Document(page_content='tion, an abscess which drains foul-smelling pus, and\npossibly fever . Symptoms for specific infections are\nas fol

In [26]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)


Fast language models, also known as efficient language models or sparse language models, have gained significant attention in the field of natural language processing (NLP) and artificial intelligence (AI) in recent years. The importance of fast language models lies in their ability to balance the two conflicting goals of:

1. **Model complexity**: To capture the rich nuances of human language, language models need to be sufficiently large and complex. This complexity is crucial for modeling human language and understanding the context of the text.
2. **Computational efficiency**: Fast language models aim to be computationally efficient, allowing for fast inference, adaptation to new tasks, and scalability to large texts. This efficiency is essential for real-world applications where processing speed and computational resources are limited.

The importance of fast language models can be summarized as follows:

1. **Scalability**: Fast language models enable processing large texts, data

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "../model/meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    low_cpu_mem_usage=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 10.04it/s]


In [9]:
from langchain_groq import ChatGroq
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="llama3-8b-8192", api_key=GROQ_API_KEY)

In [10]:
qa = RetrievalQA.from_chain_type(  
    llm=model,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs 
)  
qa.invoke(query) 

{'query': 'Tell me about fever?',
 'result': "I don't have information about fever in the context of abscesses.",
 'source_documents': [Document(page_content='tion, an abscess which drains foul-smelling pus, and\npossibly fever . Symptoms for specific infections are\nas follows:\n• Tooth and gum infections. Swollen, tender bleeding\ngums, bad breath , and pain . Severe infections may\nproduce oozing sores.\n• Throat infection. An extremely sore throat , bad breath,\na bad taste in the mouth, fever, and a sense of choking .\n• Lung infection. Chest pain, coughing, difficulty breath-\ning, fever, foul-smelling sputum, and weight loss.\n• Intraabdominal infection. Pain, fever, and possibly, if fol-\nlowing surgery, foul- smelling drainage from the wound.\n• Pelvic infection. Foul-smelling pus or blood draining\nfrom the uterus, general or localized pelvic pain, fever,and chills.\n• Skin and soft tissue infection. Infected wounds are red,\npainful, swollen, and may drain a foul-smelling pu